In [1]:
# The objective of this notebook is to extract features from the
# information source 3, which are the 3D models step files located
# in the local repository devis_path='C:/Users/BS/cpstdata/devis', and to store
# those features as npz files in a directory called
# step_path='C:/Users/BS/cpstdata/step_data'. 

# More details to be found under project proposal notebook relevant section.

In [2]:
# Importing necessary libraries
import pandas as pd
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)
import numpy as np
import sys
import glob

# Additional libraries specific to functions are imported at function definition

In [3]:
# Creating variable for path name to the extracted data
step_path='C:/Users/BS/cpstdata/step_data'

# Store this variable for usage in other notebooks
%store step_path

#Getting stored data back from original notebooks where data is generated
%store -r devis_path
%store -r cpst_path

#Checking the variables acctive in the notebook
%whos

Stored 'step_path' (str)
Variable     Type      Data/Info
--------------------------------
cpst_path    str       C:/Users/BS/cpstdata
devis_path   str       C:/Users/BS/cpstdata/devis
glob         module    <module 'glob' from 'C:\\<...>Capestone\\lib\\glob.py'>
np           module    <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
pd           module    <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
step_path    str       C:/Users/BS/cpstdata/step_data
sys          module    <module 'sys' (built-in)>


In [4]:
# Setting up for utilisation of the CAD software FreeCAD

# Importing FreeCAD
import freecad
import FreeCAD
import FreeCADGui
import Part, math
from FreeCAD import Base

# Setting path
import sys, os
import PyQt5
%gui qt5
os.environ["QT_API"] = "pyside"
prefix = os.environ['CONDA_PREFIX']
sys.path.append(prefix+"/")
sys.path.append(prefix+"lib/")

PATH_TO_FREECAD_LIBDIR not specified, using default FreeCAD version in C:/Users/BS/.conda/envs/Capestone/Library/bin


In [5]:
# Display of the version of FreeCAD used
FreeCAD.Version()

# Activate the GUI (Not used)
#FreeCADGui.showMainWindow()

# Clear the GUI selection (Not used)
#thread_func(5)
#Gui.Selection.clearSelection()
#Gui.getMainWindow().deleteLater()
#FreeCADGui.updateGui()

['0',
 '19',
 '23546 (Git)',
 'D:\x08ldgit_cachegithub.comFreeCADFreeCAD master',
 '2020/12/27 07:24:43',
 'master',
 '6b017f9a16b15b0e628c8d874c4058442dee5548']

In [6]:
# Context manager for reditrecting stdout/err to files

class redirect_output(object):
    """context manager for reditrecting stdout/err to files"""


    def __init__(self, stdout='', stderr=''):
        self.stdout = stdout
        self.stderr = stderr

    def __enter__(self):
        self.sys_stdout = sys.stdout
        self.sys_stderr = sys.stderr

        if self.stdout:
            sys.stdout = open(self.stdout, 'w')
        if self.stderr:
            if self.stderr == self.stdout:
                sys.stderr = sys.stdout
            else:
                sys.stderr = open(self.stderr, 'w')

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.sys_stdout
        sys.stderr = self.sys_stderr

In [7]:
# Function to import a file in FreeCAD and store its name

def load_step(step_file):
    print(step_file)
    FreeCAD.loadFile(step_file)
    # store file name
    variables_dict['step_file']=str(step_file)

In [8]:
# Function to select all objects of the active document, print names and labels
def objects_check():
    
    # store count of objects
    variables_dict['objects_count']=len(FreeCAD.ActiveDocument.Objects)
    
    return FreeCAD.ActiveDocument.Objects

#    # Display object label and name
#    for obj in FreeCAD.ActiveDocument.Objects:
#        print('Object Label: ',obj.Label)
#        print('Object Name: ',obj.Name)

In [9]:
# Function to select active object in the active document and store
# object label, name, haschilds

def object_check(obj):
    
    # store active object Label
    variables_dict['objects_label']=obj.Label

    # store active object Name   
    variables_dict['objects_fullname']=obj.FullName
    
    # store child check   
    variables_dict['objects_haschild']=obj.hasChildElement() 

#return FreeCAD.ActiveDocument.ActiveObject
# select the object and display selection in the GUI
#obj = App.ActiveDocument.getObject(obj.Name)
#Gui.Selection.addSelection(obj)

In [10]:
# Function to store structural information relativ to object shape

def object_features(obj):

    variables_dict['objects_childshapes_count']=\
                                len(obj.Shape.childShapes())
    
    variables_dict['objects_compounds']=\
                                len(obj.Shape.removeSplitter().Compounds)
    
    variables_dict['objects_lenght']=\
                                obj.Shape.removeSplitter().Length
    
    variables_dict['objects_ShapeType']=\
                                obj.Shape.removeSplitter().ShapeType
    
    variables_dict['objects_count_shells']=\
                                len(obj.Shape.removeSplitter().Shells)
    
    variables_dict['objects_count_solids']=\
                                len(obj.Shape.removeSplitter().Solids)
    
    variables_dict['objects_count_subshapes']=\
                                len(obj.Shape.removeSplitter().SubShapes)

In [11]:
# Function to retrieve the edges information with freeCAD

def edges_retrieve(shape):

    lenghts=[]
    curves=[]
    for edge in shape.Edges:
        lenght=edge.Length
        try:
            curve=edge.Curve
        except TypeError:
            curve='undefined curve type'
        lenghts.append(lenght)
        curves.append(curve)

    # Create a dataframe with all the edges parameters

    df=pd.DataFrame([lenghts, curves], index=['lenght','curve']).T
    df['curve']=df['curve'].astype(str)
    df['lenght']=df['lenght'].astype(float)
    df['curve']=df['curve'].str.extract(r'(?:^|(?:[.!?]\s)|<)(\w+)')
    df_g=df.groupby('curve')
    #df.head()

    # Calculate features for the edges and store in the variables list

    # - Count edges total
    variables_dict['edges_count_total']=len(df)

    # - Count edges per curve type
    for i in df.curve.unique():
        variables_dict['edges_count_{}'.format(i)]=df_g.count().loc[i,:].item()

    # - Lenght per Curve types
    for i in df.curve.unique():
        variables_dict['edges_lenght_{}'.format(i)]=df_g.sum().loc[i,:].item()

    # - Total lenght of edges
    variables_dict['edges_lenght_total']=df.lenght.sum()

    # - Lenght per Curve types / Total lenght of edges
    for i in df.curve.unique():
        variables_dict['edges_lenght_{}_to_total'.format(i)]\
            =df_g.sum().loc[i,:].item()/df.lenght.sum()

    # - Describe Edges lenght values
    for i in list(df.lenght.describe().index):
        variables_dict['edges_lenght_{}'.format(i)]=\
                                        (df.lenght.describe()).loc[i].item()

In [12]:
# Function to retrieve the faces with freeCAD

def faces_retrieve(shape):

    areas=[]
    surfaces=[]
    volumes=[]
    volume_signs=[]
    areplanar=[]
    orientations_0=[]
    orientations_1=[]
    orientations_2=[]

    for face in shape.Faces:
        area=face.Area
        surface=face.Surface
        volume=face.Volume
        if volume>0:
            volume_sign=np.abs(volume)/volume
        else:
            volume_sign=1
        isplanar=face.Surface.isPlanar()
        try:
            orientation_0=face.normalAt(0,0)[0]
            orientation_1=face.normalAt(0,0)[1]
            orientation_2=face.normalAt(0,0)[2]
        except:
            orientation_0=0
            orientation_1=0
            orientation_2=0

        areas.append(area)
        surfaces.append(surface)
        volumes.append(np.abs(volume)) # We sum absolute value of faces volumes
        volume_signs.append(volume_sign)
        areplanar.append(isplanar)
        orientations_0.append(orientation_0)
        orientations_1.append(orientation_1)
        orientations_2.append(orientation_2)

    # Create a dataframe with all the faces parameters

    df=pd.DataFrame([areas, surfaces, volumes, volume_signs,areplanar,
                     orientations_0,orientations_1,orientations_2],\
                    index=['area','surface','volume','volume_sign','planar',
                           'orientation_0', 'orientation_1', 'orientation_2']).T
    df['surface']=df['surface'].astype(str)
    df['orientation_0']=df['orientation_0'].astype(float)
    df['orientation_1']=df['orientation_1'].astype(float)
    df['orientation_2']=df['orientation_2'].astype(float)
    df[['area','volume']]=df[['area','volume']].astype(float)
    df['surface']=df['surface'].str.extract('(?:^|(?:[.!?]\s)|<)(\w+)')
    df_g=df.groupby('surface')
    #df.iloc[12:20,:]
    #df_g.count()

    # Retrieve unique faces orientations and corresponding total areas & counts

    df_orient_g=df.round(decimals=4).groupby(['orientation_0',
                                              'orientation_1',
                                              'orientation_2'])
    
    df_orient_counts=df_orient_g.count().reset_index().area.rename('quantity')
    
    df_orient_axes=df_orient_g.count().reset_index()[['orientation_0',
                                                      'orientation_1',
                                                      'orientation_2']]
    
    df_orient_area=df_orient_g.sum().reset_index().area
    
    df_orient=pd.concat([df_orient_axes,df_orient_area,df_orient_counts],axis=1)
    
    # Calculate features for the faces and store in the variables list

    # - Total area of shape
    variables_dict['shape_area_total']=shape.Area

    # - Total volume of shape
    variables_dict['shape_volume_total']=np.abs(shape.Volume)

    # - Total Count of faces
    variables_dict['faces_count_total']=len(df)

    # - Count faces per surface type
    for i in df.surface.unique():
        variables_dict['faces_count_{}'.format(i)]=\
                                            df_g.count().loc[i,'area'].item()

    # - faces area per Surface types
    for i in df.surface.unique():
        variables_dict['faces_area_{}'.format(i)]=\
                                            df_g.sum().loc[i,'area'].item()

    # - Total area of faces
    variables_dict['faces_area_total']=df.area.sum()

    # - Total volume of faces (a non-planar surface as a non-nul volume)
    variables_dict['faces_volume_total']=df.volume.sum()

    # - Area per surface types / Total area of faces
    for i in df.surface.unique():
        variables_dict['faces_area_{}_to_total'.format(i)]\
            =df_g.sum().loc[i,'area'].item()/df.area.sum()

    # - Volume per surface types / Total volume of faces
    for i in df.surface.unique():
        variables_dict['faces_volume_{}_to_total'.format(i)]\
            =df_g.sum().loc[i,'volume'].item()/df.volume.sum()

    # - Describe faces area values
    for i in list(df.area.describe().index):
        variables_dict['faces_area_{}'.format(i)]=\
                                (df.area.describe()).loc[i].item()

    # - Describe faces orientations values
    for i in list(df_orient.describe().index):
        variables_dict['faces_orient_area_{}'.format(i)]=\
                                (df_orient.area.describe()).loc[i].item()
        variables_dict['faces_orient_quantity_{}'.format(i)]=\
                                (df_orient.quantity.describe()).loc[i].item()

In [13]:
# Function to retrieve the volumes and standard bounding box information

def volumes_retrieve(shape):

    # - bounding box X
    X=shape.BoundBox.XLength
    variables_dict['boundbox_X']=X

    # - bounding box Y
    Y=shape.BoundBox.YLength
    variables_dict['boundbox_Y']=Y

    # - bounding box Z
    Z=shape.BoundBox.ZLength
    variables_dict['boundbox_Z']=Z

    # - Volume of the part
    Volume_part=np.abs(shape.Volume)
    variables_dict['volume_part']=Volume_part

    # - Volume of the bounding box
    Volume_box=X*Y*Z
    variables_dict['volume_boundbox']=Volume_box

    # - Volume of the part / Volume of the bounding box
    variables_dict['volume_ratio_part_to_boundbox']=Volume_part/Volume_box

    # - bounding box diag
    variables_dict['boundbox_diagonal']=shape.BoundBox.DiagonalLength

    # - Polynomial features of X,Y,Z

    variables_dict['boundbox_X2']=X**2
    variables_dict['boundbox_X3']=X**3
    variables_dict['boundbox_Y2']=Y**2
    variables_dict['boundbox_Y3']=Y**3
    variables_dict['boundbox_Z2']=Z**2
    variables_dict['boundbox_Z3']=Z**3
    variables_dict['boundbox_XYZ']=X*Y*Z
    variables_dict['boundbox_X2Y']=X**2*Y
    variables_dict['boundbox_X2Z']=X**2*Z
    variables_dict['boundbox_XY2']=X*Y**2
    variables_dict['boundbox_Y2Z']=Y**2*Z
    variables_dict['boundbox_XZ2']=X*Z**2
    variables_dict['boundbox_YZ2']=Y*Z**2

In [14]:
# Supporting definitions for the holes_retrieve() function

#  Copyright 2020 Ulrich Brammer <ulrich@Pauline>
#  
#  This program is free software; you can redistribute it and/or modify
#  it under the terms of the GNU General Public License as published by
#  the Free Software Foundation; either version 2 of the License, or
#  (at your option) any later version.
#  
#  This program is distributed in the hope that it will be useful,
#  but WITHOUT ANY WARRANTY; without even the implied warranty of
#  MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#  GNU General Public License for more details.
#  
#  You should have received a copy of the GNU General Public License
#  along with this program; if not, write to the Free Software
#  Foundation, Inc., 51 Franklin Street, Fifth Floor, Boston,
#  MA 02110-1301, USA.

''' To do:
Use this code to do a hole search.

actual state, growTree is the selected analyzing mechanism to build 
a tree of face nodes with characteristic data.

The tree is then analyzed to identify the type of the holes.
  
Analyzing faces step for detection of connected holes: 
- Search for largest face and start from it. 
- analyze first inner wires of a face, than outerwire.
- Stop after detection of a cylinder with a parent cylinder.
- Do not change side of part at through holes.
  
This strategy should detect first all holes on one side, 
than move to other sides. The second analyzing path can be ommitted.
  
'''

import Part, math
from FreeCAD import Base
#import DraftVecUtils

def equal_vertex(vert1, vert2, p=5):
    # compares two vertices
    return (round(vert1.X - vert2.X,p)==0 and round(vert1.Y - vert2.Y,p)==0 and round(vert1.Z - vert2.Z,p)==0)

def equal_vector(vec1, vec2, p=5):
    # compares two vectors
    return (round(vec1.x - vec2.x,p)==0 and round(vec1.y - vec2.y,p)==0 and round(vec1.z - vec2.z,p)==0)

def radial_vector(point, axis_pnt, axis):
    chord = axis_pnt.sub(point)
    norm = axis.cross(chord)
    perp = axis.cross(norm)
    # FreeCAD.Console.PrintLog( str(chord) + ' ' + str(norm) + ' ' + str(perp)+'\n')
    # print ( str(chord) + ' ' + str(norm) + ' ' + str(perp)+'\n')
    #test_line = Part.makeLine(axis_pnt.add(dist_rv),axis_pnt)
    # test_line = Part.makeLine(axis_pnt.add(perp),axis_pnt)
    # test_line = Part.makeLine(point, axis_pnt)
    # Part.show(test_line)
    return perp.normalize()

def cone_norm(point, apex, axis):
    chord = apex.sub(point)
    #norm = axis.cross(chord)
    rad = radial_vector(point, apex, axis)
    norm = rad.cross(chord)
    perp = chord.cross(norm)
    # FreeCAD.Console.PrintLog( str(chord) + ' ' + str(norm) + ' ' + str(perp)+'\n')
    # print ( str(chord) + ' ' + str(norm) + ' ' + str(perp)+'\n')
    test_line = Part.makeLine(point.add(perp),point)
    # test_line = Part.makeLine(point, axis_pnt)
    #Part.show(test_line, 'ConeNormal')
    return perp.normalize()


def cone_test(point, apex, axis, testVec, p=5):
    chord = apex.sub(point)
    norm = axis.cross(chord)
    #rad = radial_vector(point, apex, axis)
    #norm = rad.cross(chord)
    perp = chord.cross(norm)
    # FreeCAD.Console.PrintLog( str(chord) + ' ' + str(norm) + ' ' + str(perp)+'\n')
    # print ( str(chord) + ' ' + str(norm) + ' ' + str(perp)+'\n')
    test_line = Part.makeLine(point.add(perp),point)
    # test_line = Part.makeLine(point, axis_pnt)
    #Part.show(test_line, 'ConeTestNormal')
    result = testVec - perp.normalize()
    #print 'cone test result: ', result,
    #print 'cone NormVec ', testVec, ' test ', perp.normalize()
    return (round(result.x,p)==0 and round(result.y,p)==0 and round(result.z,p)==0)



class FaceTree(object):
  ''' This class defines the nodes of a tree. The tree is constructed by
  the functions growTree of the ShapeAna class.
  Each face of the part gets a node in the tree.
  The indexes are the number of the face in the original part.
  '''
  def __init__(self, unfoldShape = None, f_idx=None, Parent_node= None, Parent_edge = None, pWire = None):
    ''' unfoldShape is the Shape to be analyzed.
    '''
    self.NoStopFace = True # The analysis should only stop at a StopFace
    self.bore = False # True in case the face represents a round hole or pocket.
    if unfoldShape is not None: # fix me a shape is needed to unfold
        theFace = unfoldShape.Faces[f_idx]
    #else: #additions#print ('need a shape to unfold! Select one first!')
    self.idx = f_idx  # index of the face
    F_type = str(theFace.Surface)
    #print ('analyzing Face'+str(f_idx+1), ' ', F_type)
    
    self.node_type = None  # 'Flat' or 'Cylinder' or 'Cone', see below
    self.p_node = Parent_node   # Parent node
    self.p_edge = Parent_edge # the connecting edge to the parent node
    self.child_list = [] # List of child-nodes = link to tree structure
    self.child_idx_list = [] # List of child_idx
    # need also a list of indices of child faces
    self.normal = None # normal of the face, used to calculate the rotation, calculated below
    self.minusTan = None # flag to correct the self.secondRotAngle, Needed anymore?
    
    if Parent_node is None:
        self.rotCenter = None # A point on the edge, needed for further calculations
    else:
      curveType = str(self.p_edge.Curve)
      if "<Line" in curveType:
        edgeParameter = self.p_edge.FirstParameter
      else:
        edgeParameter = (self.p_edge.FirstParameter + self.p_edge.LastParameter) / 2.0

      self.rotCenter = self.p_edge.valueAt(edgeParameter) # A point on the edge, needed for further calculations
    
    self.cAxis = None # Axis of a cylindrical or conical face
    self.cCenter = None 
    self.radius = None # radius of a cylinder or whatever needs a radius.
    self.analysis_ok = True # indicator if something went wrong with the analysis of the face
    self.error_code = None # index to unfold_error dictionary
    
    s_Ori = theFace.Orientation


    F_type = str(theFace.Surface)
    if F_type == "<Plane object>":
      self.node_type = 'Flat' # fixme
      #FreeCAD.Console.PrintLog("Face"+ str(f_idx+1) + " Type: "+ str(self.node_type) + "\n")
    
      #s_Ori = theFace.Orientation
      #s_Axis = theFace.Surface.Axis # is not reliable in all cases
      s_Axis = theFace.Surface.normal(0.0,0.0)
      if s_Ori == 'Forward':
        self.normal = Base.Vector(s_Axis.x, s_Axis.y, s_Axis.z)
      else:                    
        self.normal = Base.Vector(-s_Axis.x, -s_Axis.y, -s_Axis.z)
      #print ('Orientation: ', s_Ori, ' Type: ', str(self.node_type), ' normal ', self.normal)

      #s_Posi = theFace.Surface.Position
      s_Posi = theFace.Edges[0].Vertexes[0].Point
      normal_line = Part.makeLine(s_Posi + self.normal, s_Posi)
      #Part.show(normal_line, 'normal_line'+str(f_idx+1)+'_')
      
      # a flat face can be a "StopFace". This is the case for a 
      # through hole. The tree growing algorithm should not go
      # through a through hole, to visit the next face.
      # This is the case when the outer wire of the actual Face has 
      # vertexes different to the parent wire.
      if Parent_node:
        if Parent_node.bore:
          #print ('child face of bore')
          # pV=Parent_edge.Vertexes[0]
          # fV = theFace.Wires[0].Vertexes[0]
          # print (pV.X, ' ', pV.Y, ' ', pV.Z)
          # print (fV.X, ' ', fV.Y, ' ', fV.Z)
          # Part.show(Parent_edge)
          # Part.show(theFace.Wires[0])
          if not equal_vertex(Parent_edge.Vertexes[0], theFace.Wires[0].Vertexes[0]):
            self.NoStopFace = False
            #print (' edge type: ', theFace.Wires[0].Edges[0])
            testEdge = theFace.Wires[0].Edges[0]
            if hasattr(testEdge,"Curve"):
              #print (theFace.Wires[0].Edges[0].__getattribute__('Curve'))
              if 'Circle' in str(testEdge.__getattribute__('Curve')):
                self.NoStopFace = True



    if F_type == "<Cylinder object>":
      self.node_type = 'Cylinder' 
      #FreeCAD.Console.PrintLog("Face"+ str(f_idx+1) + " Type: "+ str(self.node_type) + "\n")
    
      #s_Ori = self._Shape.Faces[f_idx].Orientation
      s_Center = theFace.Surface.Center
      self.cCenter = s_Center
      self.radius = theFace.Surface.Radius
      self.cAxis = theFace.Surface.Axis #fix me need to understand the minus sign?
      if Parent_node is None:
        angle_0 = theFace.ParameterRange[0]
        length_0 = theFace.ParameterRange[2]
        #s_Posi = theFace.Surface.value(angle_0, length_0)
        #s_Posi = theFace.Edges[0].Vertexes[0]
        self.rotCenter = theFace.Edges[0].Vertexes[0].Point
        # self.axis = self.cAxis
      else:
        cylinderAngle = theFace.ParameterRange[1] - theFace.ParameterRange[0]
        #print ('cylinder angle0: ', theFace.ParameterRange[0],
        #       ' cylinder angle1: ', theFace.ParameterRange[1],
        if cylinderAngle > (2*math.pi - 0.01):
          self.bore = True
          print ( 'D: ', 2*self.radius, ' L: ', abs(theFace.ParameterRange[2]-theFace.ParameterRange[3]), 
                 ' Axis: ', self.cAxis, ' center: ', s_Center)
          holes.append((2*self.radius,abs(theFace.ParameterRange[2]-theFace.ParameterRange[3])))
          if Parent_node.bore:
            print('attached 90° to above bore!')
          if Parent_node.p_node:
            if Parent_node.p_node.bore:
              print('Part of countersunk bore!')

      #self.normal = radial_vector(self.rotCenter, s_Center, self.cAxis)
      #need to check the direction of the cylinder axis
      self.u0, self.v0 = theFace.Surface.parameter(self.rotCenter)
      s_Axis = theFace.Surface.normal(self.u0, self.v0)
      if s_Ori == 'Forward':
        self.normal = Base.Vector(s_Axis.x, s_Axis.y, s_Axis.z)
      else:                    
        self.normal = Base.Vector(-s_Axis.x, -s_Axis.y, -s_Axis.z)
      
      uTest = self.u0 + 0.1
      testPnt = theFace.Surface.value(uTest, self.v0)
      tripleProd = self.normal.cross(self.cAxis).dot(testPnt-self.rotCenter)
      #print 'tripleProd: ', tripleProd
      self.tanVec = self.normal.cross(self.cAxis)
      if tripleProd < 0:
        #self.cAxis = -self.cAxis
        self.tanVec = self.tanVec.multiply(-1.0) #self.normal.cross(self.cAxis)
        self.minusTan = True
        
      #axis_line = Part.makeLine(self.rotCenter + self.normal, self.rotCenter)
      #Part.show(axis_line, 'normal_line'+str(f_idx+1)+'_')

    if F_type == "<Cone object>":
      self.node_type = 'Cone' # 
      FreeCAD.Console.PrintLog("Face"+ str(f_idx+1) + " Type: "+ str(self.node_type) + "\n")
      
      #s_Ori = self._Shape.Faces[f_idx].Orientation
      s_Center = theFace.Surface.Center
      self.cCenter = s_Center
      self.apex = theFace.Surface.Apex
      self.semiAngle = theFace.Surface.SemiAngle
      #self.radius = theFace.Surface.Radius
      self.cAxis = theFace.Surface.Axis #fix me need to understand the minus sign?
      if Parent_node is None:
        #search for an edge that is not a circle, because it could be the apex
        for edge in theFace.OuterWire.Edges:
          eType = str(edge.Curve)
          if not ("Circle" in eType):
            coneEdge = edge
            break
        self.rotCenter = coneEdge.Vertexes[0].Point
        self.axis = self.cAxis
        # test if self.rotCenter is identical with self.apex
        if equal_vector(self.rotCenter, self.apex):
          #self.p_edge.FirstParameter
          self.rotCenter = coneEdge.Vertexes[1].Point
      else:
        if equal_vector(self.rotCenter, self.apex):
          #self.p_edge.FirstParameter
          self.rotCenter = self.p_edge.valueAt(self.p_edge.LastParameter)
          self.axis = self.p_edge.tangentAt(self.p_edge.LastParameter) # the rotation axis
      #self.normal = radial_vector(self.rotCenter, s_Center, self.cAxis)
      self.normal = cone_norm(self.rotCenter, self.apex, self.cAxis)
      if cone_test(self.rotCenter, self.apex, self.cAxis, self.normal):
        #additions#print ('cone test True')
        self.reversCone = True
      else:
        #additions#print ('cone test False')
        self.reversCone = False
      #need to check the direction of the cylinder axis
      self.u0, self.v0 = theFace.Surface.parameter(self.rotCenter)
      uTest = self.u0 + 0.1
      testPnt = theFace.Surface.value(uTest, self.v0)
      tripleProd = self.normal.cross(self.cAxis).dot(testPnt-self.rotCenter)
      #additions#print ('cone tripleProd: ', tripleProd)
      #self.tanVec = self.normal.cross(self.cAxis)
      if tripleProd < 0:
        #self.tanVec = -self.tanVec #self.normal.cross(self.cAxis)
        self.minusTan = True
      self.v0 = (self.cCenter-self.apex).Length / math.cos(self.semiAngle)



  def get_Face_idx(self):
    # get the face index from the tree-element
    return self.idx

class ShapeAna(object):
  ''' This class builds the FaceTree with all information in its nodes.
  It does a shape analysis by iterative visiting of all faces of the shape.
  It does not work for shapes having inner faces not connected to the 
  outer faces.
  The algorithm is tuned to detect round holes in flat objects.
  
  '''

  def __init__(self):
    self.root = None # getObject adds the root node if parent_node == None
    self._Shape = None

    self.error_code = None
    self.failed_face_idx = None
    self.f_list = [] # list with all faces of the object
    self.idx_list = [] # list with the face indexes
    

  def getObject(self, TheShape, f_idx):
    self._Shape = TheShape.copy()

    if not self._Shape.isValid():
      FreeCAD.Console.PrintLog("The shape is not valid!" + "\n")
      self.error_code = 4  # Starting: invalid shape
      self.failed_face_idx = f_idx

    self.f_list = self._Shape.Faces[:] 
    self.idx_list = list(range(len (self._Shape.Faces)))


  def growTree(self, f_idx = 0, parent_node = None, parent_edge = None, pWire = None): 
    #take a face
    # analyze the face and create a node object of class FaceTree.
    tNode = FaceTree(self._Shape, f_idx, parent_node, parent_edge, pWire)
    if tNode.NoStopFace:
      # This face should be a node in the tree, and is therefore known!
      # removed from the list of all not visited faces
      self.idx_list.remove(f_idx)
      # This means, it could also not be found as neighbor face anymore.
  
      if parent_node is None:
        self.root = tNode
      else:
        #put the node of the face into the tree structure.
        parent_node.child_list.append(tNode)
        
      #print ('idx_list: ', self.idx_list)
      such_list = self.idx_list[:] 
  
      if self.error_code is None: # go only further in case of no error.
        edge_list = []
        #print (self.f_list[f_idx].Area)
        #print 'Face', f_idx+1, ' has ', len(self.f_list[f_idx].OuterWire.Edges), 'edges.'
        # Each face has a list of wires.
        # Analyze first the inner wires, than the outer wire.
        # We therefore must rearrange the list of wires, because the outer
        # wire is the first in the list.
        wireList = list(range(len(self.f_list[f_idx].Wires)))
        wireList.remove(0)
        wireList.append(0)
        #for fWire in self.f_list[f_idx].Wires:
        for wIdx in wireList:
          fWire = self.f_list[f_idx].Wires[wIdx]
          for n_edge in fWire.Edges:
                if parent_edge:
                    if not parent_edge.isSame(n_edge):
                        edge_list.append(n_edge)
                else:
                    edge_list.append(n_edge)
          #print 'edge_list edges: ', len(edge_list)
    
          #für jede Kante:
          for sEdge in edge_list:
            #suche die angrenzende Fläche.
            the_index = None
            for i in self.idx_list:
              for sf_edge in self.f_list[i].Edges:
                if sf_edge.isSame(sEdge):
                    the_index = i
                    #print 'got edge face: Face', str(i+1)
                    break
              if the_index is not None:
                #Baumzweig(angrenzende Fläche)
                self.growTree(the_index, tNode, sEdge, fWire)
                break
            #print 'got face index: ', the_index

In [15]:
# Function to retrieve holes parameters

def holes_retrieve(shape):
    print ('Starting with face1')
    theFace = 0
    #s = Gui.Selection.getSelection()
    #s = FreeCAD.ActiveDocument.ActiveObject

    theUnfold = ShapeAna()
    theUnfold.getObject(shape, theFace)#s[0]
    theUnfold.growTree(theFace, None, None)

    FreeCAD.Console.PrintLog('Analyse beendet' + "\n")

    # List of the holes identified
    print(holes)

    # - Number of holes
    variables_dict['holes_count']=len(holes)
    
    if len(holes)>0:
    
        # - Number of different holes diameters
        variables_dict['holes_diameters_count']=\
                                    len(np.unique([i[0] for i in holes]))

        # - Diameters: mini maxi average std
        variables_dict['holes_diameter_min']=min([i[0] for i in holes])
        variables_dict['holes_diameter_max']=max([i[0] for i in holes])
        variables_dict['holes_diameter_avg']=np.mean([i[0] for i in holes])
        variables_dict['holes_diameter_std']=np.std([i[0] for i in holes])

        # - Volume of drillings
        variables_dict['holes_volume_min']=\
                                min([i[1]*np.pi*(i[0]/2)**2 for i in holes])
        variables_dict['holes_volume_max']=\
                                max([i[1]*np.pi*(i[0]/2)**2 for i in holes])
        variables_dict['holes_volume_avg']=\
                                np.mean([i[1]*np.pi*(i[0]/2)**2 for i in holes])
        variables_dict['holes_volume_std']=\
                                np.std([i[1]*np.pi*(i[0]/2)**2 for i in holes])
    else:
        variables_dict['holes_diameters_count']=np.nan
        variables_dict['holes_diameter_min']=np.nan
        variables_dict['holes_diameter_max']=np.nan
        variables_dict['holes_diameter_avg']=np.nan
        variables_dict['holes_diameter_std']=np.nan
        variables_dict['holes_volume_min']=np.nan
        variables_dict['holes_volume_max']=np.nan
        variables_dict['holes_volume_avg']=np.nan
        variables_dict['holes_volume_std']=np.nan


In [16]:
# Supporting definitions for the box_retrieve() function
# adapted from: http://jamesgregson.blogspot.com/2011/03/latex-test.html

from numpy import ndarray, array, asarray, dot, cross, cov,
from numpy import array, finfo, min as npmin, max as npmax

from numpy.linalg import eigh, norm

class OBB:
    def __init__(self):
        self.rotation = None
        self.min = None
        self.max = None

    def transform(self, point):
        return dot(array(point), self.rotation)

    @property
    def centroid(self):
        return self.transform((self.min + self.max) / 2.0)

    @property
    def extents(self):
        return abs(self.transform((self.max - self.min) / 2.0))

    @property
    def points(self):
        return [
            # upper cap: ccw order in a right-hand system
            # rightmost, topmost, farthest
            self.transform((self.max[0], self.max[1], self.min[2])),
            # leftmost, topmost, farthest
            self.transform((self.min[0], self.max[1], self.min[2])),
            # leftmost, topmost, closest
            self.transform((self.min[0], self.max[1], self.max[2])),
            # rightmost, topmost, closest
            self.transform(self.max),
            # lower cap: cw order in a right-hand system
            # leftmost, bottommost, farthest
            self.transform(self.min),
            # rightmost, bottommost, farthest
            self.transform((self.max[0], self.min[1], self.min[2])),
            # rightmost, bottommost, closest
            self.transform((self.max[0], self.min[1], self.max[2])),
            # leftmost, bottommost, closest
            self.transform((self.min[0], self.min[1], self.max[2])),
        ]

    @classmethod
    def build_from_covariance_matrix(cls, covariance_matrix, points):
        if not isinstance(points, ndarray):
            points = array(points, dtype=float)
        assert points.shape[1] == 3

        obb = OBB()

        _, eigen_vectors = eigh(covariance_matrix)

        def try_to_normalize(v):
            n = norm(v)
            if n < finfo(float).resolution:
                raise ZeroDivisionError
            return v / n

        r = try_to_normalize(eigen_vectors[:, 0])
        u = try_to_normalize(eigen_vectors[:, 1])
        f = try_to_normalize(eigen_vectors[:, 2])

        obb.rotation = array((r, u, f)).T

        # apply the rotation to all the position vectors of the array
        # TODO : this operation could be vectorized with tensordot
        p_primes = asarray([obb.rotation.dot(p) for p in points])
        obb.min = npmin(p_primes, axis=0)
        obb.max = npmax(p_primes, axis=0)

        return obb

    @classmethod
    def build_from_triangles(cls, points, triangles):
        for point in points:
            if len(point) != 3:
                raise Exception('points have to have 3-elements')

        weighed_mean = array([0, 0, 0], dtype=float)
        area_sum = 0
        c00 = c01 = c02 = c11 = c12 = c22 = 0
        for i in range(0, len(triangles), 3):
            p = array(points[triangles[i]], dtype=float)
            q = array(points[triangles[i + 1]], dtype=float)
            r = array(points[triangles[i + 2]], dtype=float)
            mean = (p + q + r) / 3.0
            area = norm(cross((q - p), (r - p))) / 2.0
            weighed_mean += mean * area
            area_sum += area
            c00 += (9.0 * mean[0] * mean[0] + p[0] * p[0] + q[0] * q[0] + r[0] * r[0]) * (area / 12.0)
            c01 += (9.0 * mean[0] * mean[1] + p[0] * p[1] + q[0] * q[1] + r[0] * r[1]) * (area / 12.0)
            c02 += (9.0 * mean[0] * mean[2] + p[0] * p[2] + q[0] * q[2] + r[0] * r[2]) * (area / 12.0)
            c11 += (9.0 * mean[1] * mean[1] + p[1] * p[1] + q[1] * q[1] + r[1] * r[1]) * (area / 12.0)
            c12 += (9.0 * mean[1] * mean[2] + p[1] * p[2] + q[1] * q[2] + r[1] * r[2]) * (area / 12.0)

        weighed_mean /= area_sum
        c00 /= area_sum
        c01 /= area_sum
        c02 /= area_sum
        c11 /= area_sum
        c12 /= area_sum
        c22 /= area_sum

        c00 -= weighed_mean[0] * weighed_mean[0]
        c01 -= weighed_mean[0] * weighed_mean[1]
        c02 -= weighed_mean[0] * weighed_mean[2]
        c11 -= weighed_mean[1] * weighed_mean[1]
        c12 -= weighed_mean[1] * weighed_mean[2]
        c22 -= weighed_mean[2] * weighed_mean[2]

        covariance_matrix = ndarray(shape=(3, 3), dtype=float)
        covariance_matrix[0, 0] = c00
        covariance_matrix[0, 1] = c01
        covariance_matrix[0, 2] = c02
        covariance_matrix[1, 0] = c01
        covariance_matrix[1, 1] = c11
        covariance_matrix[1, 2] = c12
        covariance_matrix[2, 0] = c02
        covariance_matrix[1, 2] = c12
        covariance_matrix[2, 2] = c22

        return OBB.build_from_covariance_matrix(covariance_matrix, points)

    @classmethod
    def build_from_points(cls, points):
        if not isinstance(points, ndarray):
            points = array(points, dtype=float)
        assert points.shape[1] == 3, 'points have to have 3-elements'
        # no need to store the covariance matrix
        return OBB.build_from_covariance_matrix(cov(points, y=None,
                                                    rowvar=0, bias=1), points)

In [17]:
# Function to retrieve the optimal bounding box around the object,
# with optimized orientation

from scipy.spatial import ConvexHull
import Points, Part
import pyobb
#import PointsGui
#Gui.runCommand('Points_Convert')

def box_retrieve(shape):
    points=[v.Point for v in shape.Vertexes]
    hull = ConvexHull(points,qhull_options=None)#'QJ'
    faces = []
    for i in hull.simplices:
        wire = Part.makePolygon([FreeCAD.Vector(points[i[0]]),
                                 FreeCAD.Vector(points[i[1]]),
                                 FreeCAD.Vector(points[i[2]]),
                                 FreeCAD.Vector(points[i[0]])])
        faces.append(Part.Face(wire))

    shell = Part.makeShell(faces)
    Part.show(shell)

    hullpts = [points[i] for i in hull.vertices]
    obb = OBB.build_from_points(hullpts)

    obbvec = [FreeCAD.Vector(p)for p in obb.points]

    faces = []
    idx   = [[0,1,2,3,0],[4,5,6,7,4],[0,1,4,5,0],
             [2,3,6,7,2],[1,2,7,4,1],[0,5,6,3,0]]
    for ix in idx:
        wire = Part.makePolygon([obbvec[i] for i in ix])
        faces.append(Part.Face(wire))

    shell = Part.makeShell(faces)
    doc = FreeCAD.ActiveDocument
    obj = doc.addObject('Part::Feature', 'OBB')
    obj.Shape = shell
    #obj.ViewObject.ShapeColor = (1.00,0.67,0.00)
    #obj.ViewObject.Transparency = 50

    shell_1 = Part.makeShell(faces)
    obj_1 = doc.addObject('Part::Feature', 'BBOX')
    obj_1.Shape = shell_1
    #obj_1.ViewObject.ShapeColor = (1.00,0.67,0.00)
    #obj_1.ViewObject.Transparency = 50


    # Volumes and bounding box / Optimized BOX

    # - Volume of the optimized bounding box
    Volume_box_opt=np.abs(obj.Shape.Volume)
    variables_dict['boundbox_optimized_volume']=Volume_box_opt

    # - Volume of the part / Volume of the bounding box
    variables_dict['boundbox_optimized_volume_ratio']=\
                                variables_dict['volume_part']/Volume_box_opt

In [18]:
# Defining a waiting time to allow the GUI to load the step file
# Function is not used currently, as we are not using the GUI.

import threading
import time

exit_flag = threading.Event()

def thread_func(sec): 
    while not exit_flag.wait(timeout=sec):
        print('{} have passed ... we continue'.format(sec))
        break

In [19]:
# Identify the list of step files to be processed 

step_list=\
        glob.glob(devis_path+'/**/*.step', recursive=True) \
        +glob.glob(devis_path+'/**/*.stp', recursive=True)

step_files=[i.replace('\\','/') for i in step_list]

step_files[0:10]

['C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210809aa000avantclebuteebague.step',
 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210810aa000mancheclebuteebague.step',
 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d210024aa000pionantirotation/d210024aa000pionantirotation.step',
 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d210047aa000soclesupportfiltreir/d210047aa000soclesupportfiltreir.step',
 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d210048aa000couverclesupportfiltreir/d210048aa000couverclesupportfiltreir.step',
 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d220013aa000pontthermique/d220013aa000pontthermique.step',
 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d220015ae000boitiersystemeoptique/d220015aa000boitiersystemeoptique.step',
 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059

In [20]:
# Function to preprocess step files by batches

def get_batch(step_files,size):
    idx_shuff=np.arange(len(step_files))
    #np.random.shuffle(idx_shuff)
    for i in range(0,len(step_files),size):
        idx_batch=idx_shuff[i:i+size]
        yield [step_files[i] for i in idx_batch]

for step_files_batch in get_batch(step_files, 3):
    print(step_files_batch)

['C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210809aa000avantclebuteebague.step', 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210810aa000mancheclebuteebague.step', 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d210024aa000pionantirotation/d210024aa000pionantirotation.step']
['C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d210047aa000soclesupportfiltreir/d210047aa000soclesupportfiltreir.step', 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d210048aa000couverclesupportfiltreir/d210048aa000couverclesupportfiltreir.step', 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d220013aa000pontthermique/d220013aa000pontthermique.step']
['C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d220015ae000boitiersystemeoptique/d220015aa000boitiersystemeoptique.step', 'C:/Users/BS/cpstdata/devis/2020/c0000007scrome/202005902020

In [21]:
# Starting iteration on the list of steps files to be processed

# Working by batches of files
batch_size=1
batches=np.int(np.ceil(len(step_files)/batch_size))
print('batch size: ',batch_size)
print('batches: ',batches)
skipped_batch=0

for batch,step_files_batch in enumerate(get_batch(step_files, batch_size)):
    
    print('\n\nWorking on batch {} from {} ...'.format(batch+1,batches))
    print('Files in batch: ',step_files_batch)
    
    with redirect_output(step_path+'/output_{}_from_{}.txt'\
                                                     .format(batch+1,batches)):
        
        # Initialize dataframe containing variables extracted from step files
        df_features=pd.DataFrame()
        
        try:

            for step_file in step_files_batch:

                # Initialize a dictionary of variables to be extracted
                print('\n\nInitialize dictionnary of variables ...')
                variables_dict={}

                # Load step file
                print('Loading new step file.')
                load_step(step_file)

                # Get list of objects in step file
                print('Get list of objects in step file...')
                objects=objects_check()

                # Iterate over the objects
                print('Iterate over the objects...')
                for obj in objects:

                    print('Adressing next object...')
                    obj_volume=obj.Shape.Volume
                    print('object volume: ',obj_volume)

                    # Store information about each object
                    print('Store information about each object...')
                    object_check(obj)

                    # Store structural information relativ to object shape
                    print('Store information relativ to object shape...')
                    object_features(obj)

                    # Get list of solids in object
                    print('Get list of solids in object...')
                    solids=obj.Shape.removeSplitter().Solids
                    #if obj.hasChildElement(): solids=obj.Shape.childShapes()
                    #else: solids=[obj.Shape]

                    # Iterate over the solids
                    print('Iterate over the solids...')
                    for solid in solids:

                        print('Adressing next solid...')

                        try:
                            # Refine shape to produces a non-parametric copy
                            # with a refined shape,
                            # that is, with certain edges and faces cleaned up
                            print('Remove splitter')
                            shape=solid.removeSplitter()

                            print('Get volumes')
                            shape_volume=shape.Volume

                            # Work only on shapes with certain volume ratio
                            # to the main assembly
                            print('Individual solid shape volume: {:.1f} \n(Object shape volume: {:.1f})'\
                                      .format(shape_volume,obj_volume))

                            if shape_volume > 0.10 * obj_volume:

                                print('Individual solid considered in the extraction, adressing solid...')
                                try:
                                    # Check for working only on valid shapes
                                    print('Check validity...')
                                    shape.sewShape()
                                    shape.check()

                                    # Retrive edges information
                                    print('Edges...')
                                    edges_retrieve(shape)

                                    # Retrieve faces information
                                    print('Faces...')
                                    faces_retrieve(shape)

                                    # Retrieve volumes and bounding box
                                    print('Volumes...')
                                    volumes_retrieve(shape)

                                    # Retrieve holes:
                                    print('Holes...')
                                    holes=[]
                                    holes_retrieve(shape)

                                    # Generate optimal oriented box
                                    # around object:
                                    try:
                                        print('Box...')
                                        box_retrieve(shape)
                                    except:
                                        print('Error in box retrieval: skipped.')
                                        variables_dict['boundbox_optimized_volume']=np.nan
                                        variables_dict['boundbox_optimized_volume_ratio']=np.nan
                                    print('Features extracted.')
                                    # Create DataFrame of features for a single step file from the dictionnary
                                    df_feature=pd.DataFrame.from_dict({step_file: variables_dict}).T
                                    
                                    # Register the information on file processed in the dataframe
                                    path_list=os.path.normpath(step_file).split(os.path.sep)
                                    print(path_list)
                                    for i in range(len(path_list)):
                                        df_feature['path_{}'.format(i)]=path_list[i]
                                    df_feature['file_name']=os.path.basename(step_file)
                                    df_feature['step_file']=step_file
                                    display(df_feature)
                                    df_features=pd.concat([df_features, df_feature], axis=0).reset_index(drop=True)
                                except:
                                    print('Error in shape check: current shape skipped.')
                            else:
                                print('Shape too small, is skipped.')
                        except AttributeError:
                            print('Issue with solid, is skipped.')

                    FreeCAD.closeActiveTransaction()

            # Save dataframe as .npz file
            np.savez(step_path+'/step_data_{}_from_{}.npz'\
                     .format(batch+1, batches), features=df_features,
                     names=df_features.columns)
            print('npz file for this batch generated')
        
        except dummy_error:
            print('Error in batch: current batch skipped.')
            skipped_batch+=1

print('Done')
print('Number of batch skipped: ',skipped_batch)

batch size:  1
batches:  94


Working on batch 1 from 94 ...
Files in batch:  ['C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210809aa000avantclebuteebague.step']


,boundbox_X,boundbox_X2,boundbox_X2Y,boundbox_X2Z,boundbox_X3,boundbox_XY2,boundbox_XYZ,boundbox_XZ2,boundbox_Y,boundbox_Y2,boundbox_Y2Z,boundbox_Y3,boundbox_YZ2,boundbox_Z,boundbox_Z2,boundbox_Z3,boundbox_diagonal,boundbox_optimized_volume,boundbox_optimized_volume_ratio,edges_count_BSplineCurve,edges_count_Circle,edges_count_Line,edges_count_total,edges_lenght_25%,edges_lenght_50%,edges_lenght_75%,edges_lenght_BSplineCurve,edges_lenght_BSplineCurve_to_total,edges_lenght_Circle,edges_lenght_Circle_to_total,edges_lenght_Line,edges_lenght_Line_to_total,edges_lenght_count,edges_lenght_max,edges_lenght_mean,edges_lenght_min,edges_lenght_std,edges_lenght_total,faces_area_25%,faces_area_50%,faces_area_75%,faces_area_BSplineSurface,faces_area_BSplineSurface_to_total,faces_area_Cone,faces_area_Cone_to_total,faces_area_Cylinder,faces_area_Cylinder_to_total,faces_area_Plane,faces_area_Plane_to_total,faces_area_count,faces_area_max,faces_area_mean,faces_area_min,faces_area_std,faces_area_total,faces_count_BSplineSurface,faces_count_Cone,faces_count_Cylinder,faces_count_Plane,faces_count_total,faces_orient_area_25%,faces_orient_area_50%,faces_orient_area_75%,faces_orient_area_count,faces_orient_area_max,faces_orient_area_mean,faces_orient_area_min,faces_orient_area_std,faces_orient_quantity_25%,faces_orient_quantity_50%,faces_orient_quantity_75%,faces_orient_quantity_count,faces_orient_quantity_max,faces_orient_quantity_mean,faces_orient_quantity_min,faces_orient_quantity_std,faces_volume_BSplineSurface_to_total,faces_volume_Cone_to_total,faces_volume_Cylinder_to_total,faces_volume_Plane_to_total,faces_volume_total,holes_count,holes_diameter_avg,holes_diameter_max,holes_diameter_min,holes_diameter_std,holes_diameters_count,holes_volume_avg,holes_volume_max,holes_volume_min,holes_volume_std,objects_ShapeType,objects_childshapes_count,objects_compounds,objects_count,objects_count_shells,objects_count_solids,objects_count_subshapes,objects_fullname,objects_haschild,objects_label,objects_lenght,shape_area_total,shape_volume_total,step_file,volume_boundbox,volume_part,volume_ratio_part_to_boundbox,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,file_name
C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210809aa000avantclebuteebague.step,22.1,488.41,4884.1,17680.4,10793.9,2210,8000.2,28960.7,10,100,3620,1000,13104.4,36.2,1310.44,47437.9,43.5758,7200.11,0.294379,13,32,40,85,2,3.74992,7.86487,72.8445,0.104886,507.681,0.730988,113.988,0.164127,85,56.8628,8.17075,0.00279521,14.1615,694.514,2.4658,18.8,48.3927,5.86455,0.00325631,95.6997,0.0531375,1235.2,0.685847,464.22,0.257759,34,510.83,52.97,0.848528,111.864,1800.98,1,8,9,16,34,5.8646,49.5468,191.116,9,817.35,200.109,1.697,280.349,2,3,6,9,9,3.77778,1,2.72845,0.000809261,0.0747922,0.924399,2.05624e-17,4470.18,5,2.8,5,1.5,1.28841,5,35.338,77.7296,7.06858,26.4403,Solid,1,0,1,1,1,1,Unnamed#Solid,False,D21-0809-AA-000_AVANT_CLE_BUTEE_BAGUE,1389.03,1800.98,2119.56,C:/Users/BS/cpstdata/devis/2020/c0000007scrome...,8000.2,2119.56,0.264938,C:,Users,BS,cpstdata,devis,2020,c0000007scrome,2020058820200326,d220808aa000clebuteebague,d210809aa000avantclebuteebague.step,d210809aa000avantclebuteebague.step




Working on batch 2 from 94 ...
Files in batch:  ['C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210810aa000mancheclebuteebague.step']


,boundbox_X,boundbox_X2,boundbox_X2Y,boundbox_X2Z,boundbox_X3,boundbox_XY2,boundbox_XYZ,boundbox_XZ2,boundbox_Y,boundbox_Y2,boundbox_Y2Z,boundbox_Y3,boundbox_YZ2,boundbox_Z,boundbox_Z2,boundbox_Z3,boundbox_diagonal,boundbox_optimized_volume,boundbox_optimized_volume_ratio,edges_count_BSplineCurve,edges_count_Circle,edges_count_Line,edges_count_total,edges_lenght_25%,edges_lenght_50%,edges_lenght_75%,edges_lenght_BSplineCurve,edges_lenght_BSplineCurve_to_total,edges_lenght_Circle,edges_lenght_Circle_to_total,edges_lenght_Line,edges_lenght_Line_to_total,edges_lenght_count,edges_lenght_max,edges_lenght_mean,edges_lenght_min,edges_lenght_std,edges_lenght_total,faces_area_25%,faces_area_50%,faces_area_75%,faces_area_Cone,faces_area_Cone_to_total,faces_area_Cylinder,faces_area_Cylinder_to_total,faces_area_Plane,faces_area_Plane_to_total,faces_area_count,faces_area_max,faces_area_mean,faces_area_min,faces_area_std,faces_area_total,faces_count_Cone,faces_count_Cylinder,faces_count_Plane,faces_count_total,faces_orient_area_25%,faces_orient_area_50%,faces_orient_area_75%,faces_orient_area_count,faces_orient_area_max,faces_orient_area_mean,faces_orient_area_min,faces_orient_area_std,faces_orient_quantity_25%,faces_orient_quantity_50%,faces_orient_quantity_75%,faces_orient_quantity_count,faces_orient_quantity_max,faces_orient_quantity_mean,faces_orient_quantity_min,faces_orient_quantity_std,faces_volume_Cone_to_total,faces_volume_Cylinder_to_total,faces_volume_Plane_to_total,faces_volume_total,holes_count,holes_diameter_avg,holes_diameter_max,holes_diameter_min,holes_diameter_std,holes_diameters_count,holes_volume_avg,holes_volume_max,holes_volume_min,holes_volume_std,objects_ShapeType,objects_childshapes_count,objects_compounds,objects_count,objects_count_shells,objects_count_solids,objects_count_subshapes,objects_fullname,objects_haschild,objects_label,objects_lenght,shape_area_total,shape_volume_total,step_file,volume_boundbox,volume_part,volume_ratio_part_to_boundbox,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,file_name
C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020058820200326/d220808aa000clebuteebague/d210810aa000mancheclebuteebague.step,8,64,512,3840,512,512,3840,28800,8,64,3840,512,28800,60,3600,216000,61.0574,NaN,NaN,2,12,6,20,1.41421,9.42478,11.3883,58,0.306359,125.664,0.663762,5.65685,0.0298798,20,54.5,9.46603,0.707107,11.5816,189.321,8.33041,15.5501,33.7721,59.9789,0.0386013,1435.71,0.923994,58.1195,0.0374046,10,1369.73,155.381,6.10896,427.091,1553.81,6,2,2,10,19.635,38.4845,43.3182,5,1435.71,310.761,16.6608,628.97,1,2,2,5,4,2,1,1.22474,0.0248904,0.97511,0,1940.59,2,7,8,6,1,2,1419.21,2739.47,98.9602,1320.25,Solid,1,0,1,1,1,1,Unnamed1#Solid,False,D21-0810-AA-000_MANCHE_CLE_BUTEE_BAGUE,378.641,1553.81,2911.21,C:/Users/BS/cpstdata/devis/2020/c0000007scrome...,3840,2911.21,0.758127,C:,Users,BS,cpstdata,devis,2020,c0000007scrome,2020058820200326,d220808aa000clebuteebague,d210810aa000mancheclebuteebague.step,d210810aa000mancheclebuteebague.step




Working on batch 3 from 94 ...
Files in batch:  ['C:/Users/BS/cpstdata/devis/2020/c0000007scrome/2020059020200327/d210024aa000pionantirotation/d210024aa000pionantirotation.step']


...

,boundbox_X,boundbox_X2,boundbox_X2Y,boundbox_X2Z,boundbox_X3,boundbox_XY2,boundbox_XYZ,boundbox_XZ2,boundbox_Y,boundbox_Y2,boundbox_Y2Z,boundbox_Y3,boundbox_YZ2,boundbox_Z,boundbox_Z2,boundbox_Z3,boundbox_diagonal,boundbox_optimized_volume,boundbox_optimized_volume_ratio,edges_count_BSplineCurve,edges_count_Circle,edges_count_Line,edges_count_total,edges_lenght_25%,edges_lenght_50%,edges_lenght_75%,edges_lenght_BSplineCurve,edges_lenght_BSplineCurve_to_total,edges_lenght_Circle,edges_lenght_Circle_to_total,edges_lenght_Line,edges_lenght_Line_to_total,edges_lenght_count,edges_lenght_max,edges_lenght_mean,edges_lenght_min,edges_lenght_std,edges_lenght_total,faces_area_25%,faces_area_50%,faces_area_75%,faces_area_Cone,faces_area_Cone_to_total,faces_area_Cylinder,faces_area_Cylinder_to_total,faces_area_Plane,faces_area_Plane_to_total,faces_area_count,faces_area_max,faces_area_mean,faces_area_min,faces_area_std,faces_area_total,faces_count_Cone,faces_count_Cylinder,faces_count_Plane,faces_count_total,faces_orient_area_25%,faces_orient_area_50%,faces_orient_area_75%,faces_orient_area_count,faces_orient_area_max,faces_orient_area_mean,faces_orient_area_min,faces_orient_area_std,faces_orient_quantity_25%,faces_orient_quantity_50%,faces_orient_quantity_75%,faces_orient_quantity_count,faces_orient_quantity_max,faces_orient_quantity_mean,faces_orient_quantity_min,faces_orient_quantity_std,faces_volume_Cone_to_total,faces_volume_Cylinder_to_total,faces_volume_Plane_to_total,faces_volume_total,holes_count,holes_diameter_avg,holes_diameter_max,holes_diameter_min,holes_diameter_std,holes_diameters_count,holes_volume_avg,holes_volume_max,holes_volume_min,holes_volume_std,objects_ShapeType,objects_childshapes_count,objects_compounds,objects_count,objects_count_shells,objects_count_solids,objects_count_subshapes,objects_fullname,objects_haschild,objects_label,objects_lenght,shape_area_total,shape_volume_total,step_file,volume_boundbox,volume_part,volume_ratio_part_to_boundbox,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,file_name
C:/Users/BS/cpstdata/devis/2021/c0000028amplitudelisses/2021002520210212/pm2010818arefb40037049couvercle1/pm2010818a.step,6.5,42.25,338,274.625,274.625,416,338,274.625,8,64,416,512,338,6.5,42.25,274.625,12.1861,84.5,2.12884,10,24,20,54,0.821211,1.74413,4.80745,14.7578,0.0921156,114.759,0.716304,30.6931,0.19158,54,10.2102,2.96685,0.535987,2.89766,160.21,3.04862,4.46012,5.63686,32.9345,0.117354,203.3,0.724408,44.4085,0.158238,19,137.328,14.7707,2.14493,32.9144,280.643,8,2,9,19,11.1461,14.5866,30.3241,8,137.327,35.0804,8.9202,45.4065,1,2,4,8,4,2.375,1,1.40789,0.0905826,0.909417,3.79325e-16,202.788,2,4.871,6.5,3.242,1.629,2,147.359,234.936,59.7825,87.5768,Solid,1,0,10,1,1,1,Unnamed93#Solid006,False,Solid006,320.42,280.643,179.887,C:/Users/BS/cpstdata/devis/2021/c0000028amplit...,338,179.887,0.53221,C:,Users,BS,cpstdata,devis,2021,c0000028amplitudelisses,2021002520210212,pm2010818arefb40037049couvercle1,pm2010818a.step,pm2010818a.step


,boundbox_X,boundbox_X2,boundbox_X2Y,boundbox_X2Z,boundbox_X3,boundbox_XY2,boundbox_XYZ,boundbox_XZ2,boundbox_Y,boundbox_Y2,boundbox_Y2Z,boundbox_Y3,boundbox_YZ2,boundbox_Z,boundbox_Z2,boundbox_Z3,boundbox_diagonal,boundbox_optimized_volume,boundbox_optimized_volume_ratio,edges_count_BSplineCurve,edges_count_Circle,edges_count_Line,edges_count_total,edges_lenght_25%,edges_lenght_50%,edges_lenght_75%,edges_lenght_BSplineCurve,edges_lenght_BSplineCurve_to_total,edges_lenght_Circle,edges_lenght_Circle_to_total,edges_lenght_Line,edges_lenght_Line_to_total,edges_lenght_count,edges_lenght_max,edges_lenght_mean,edges_lenght_min,edges_lenght_std,edges_lenght_total,faces_area_25%,faces_area_50%,faces_area_75%,faces_area_Cone,faces_area_Cone_to_total,faces_area_Cylinder,faces_area_Cylinder_to_total,faces_area_Plane,faces_area_Plane_to_total,faces_area_count,faces_area_max,faces_area_mean,faces_area_min,faces_area_std,faces_area_total,faces_count_Cone,faces_count_Cylinder,faces_count_Plane,faces_count_total,faces_orient_area_25%,faces_orient_area_50%,faces_orient_area_75%,faces_orient_area_count,faces_orient_area_max,faces_orient_area_mean,faces_orient_area_min,faces_orient_area_std,faces_orient_quantity_25%,faces_orient_quantity_50%,faces_orient_quantity_75%,faces_orient_quantity_count,faces_orient_quantity_max,faces_orient_quantity_mean,faces_orient_quantity_min,faces_orient_quantity_std,faces_volume_Cone_to_total,faces_volume_Cylinder_to_total,faces_volume_Plane_to_total,faces_volume_total,holes_count,holes_diameter_avg,holes_diameter_max,holes_diameter_min,holes_diameter_std,holes_diameters_count,holes_volume_avg,holes_volume_max,holes_volume_min,holes_volume_std,objects_ShapeType,objects_childshapes_count,objects_compounds,objects_count,objects_count_shells,objects_count_solids,objects_count_subshapes,objects_fullname,objects_haschild,objects_label,objects_lenght,shape_area_total,shape_volume_total,step_file,volume_boundbox,volume_part,volume_ratio_part_to_boundbox,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,file_name
C:/Users/BS/cpstdata/devis/2021/c0000028amplitudelisses/2021002520210212/pm2010818arefb40037049couvercle1/pm2010818a.step,6.5,42.25,338,274.625,274.625,416,338,274.625,8,64,416,512,338,6.5,42.25,274.625,12.1861,84.5,2.12884,10,24,20,54,0.821211,1.74413,4.80745,14.7578,0.0921156,114.759,0.716304,30.6931,0.19158,54,10.2102,2.96685,0.535987,2.89766,160.21,3.04862,4.46012,5.63686,32.9345,0.117354,203.3,0.724408,44.4085,0.158238,19,137.328,14.7707,2.14493,32.9144,280.643,8,2,9,19,11.1461,14.5866,30.3241,8,137.327,35.0804,8.9202,45.4065,1,2,4,8,4,2.375,1,1.40789,0.0905826,0.909417,3.79325e-16,202.788,2,4.871,6.5,3.242,1.629,2,147.359,234.936,59.7825,87.5768,Solid,1,0,10,1,1,1,Unnamed93#Solid007,False,Solid007,320.42,280.643,179.887,C:/Users/BS/cpstdata/devis/2021/c0000028amplit...,338,179.887,0.53221,C:,Users,BS,cpstdata,devis,2021,c0000028amplitudelisses,2021002520210212,pm2010818arefb40037049couvercle1,pm2010818a.step,pm2010818a.step


,boundbox_X,boundbox_X2,boundbox_X2Y,boundbox_X2Z,boundbox_X3,boundbox_XY2,boundbox_XYZ,boundbox_XZ2,boundbox_Y,boundbox_Y2,boundbox_Y2Z,boundbox_Y3,boundbox_YZ2,boundbox_Z,boundbox_Z2,boundbox_Z3,boundbox_diagonal,boundbox_optimized_volume,boundbox_optimized_volume_ratio,edges_count_BSplineCurve,edges_count_Circle,edges_count_Line,edges_count_total,edges_lenght_25%,edges_lenght_50%,edges_lenght_75%,edges_lenght_BSplineCurve,edges_lenght_BSplineCurve_to_total,edges_lenght_Circle,edges_lenght_Circle_to_total,edges_lenght_Line,edges_lenght_Line_to_total,edges_lenght_count,edges_lenght_max,edges_lenght_mean,edges_lenght_min,edges_lenght_std,edges_lenght_total,faces_area_25%,faces_area_50%,faces_area_75%,faces_area_Cone,faces_area_Cone_to_total,faces_area_Cylinder,faces_area_Cylinder_to_total,faces_area_Plane,faces_area_Plane_to_total,faces_area_count,faces_area_max,faces_area_mean,faces_area_min,faces_area_std,faces_area_total,faces_count_Cone,faces_count_Cylinder,faces_count_Plane,faces_count_total,faces_orient_area_25%,faces_orient_area_50%,faces_orient_area_75%,faces_orient_area_count,faces_orient_area_max,faces_orient_area_mean,faces_orient_area_min,faces_orient_area_std,faces_orient_quantity_25%,faces_orient_quantity_50%,faces_orient_quantity_75%,faces_orient_quantity_count,faces_orient_quantity_max,faces_orient_quantity_mean,faces_orient_quantity_min,faces_orient_quantity_std,faces_volume_Cone_to_total,faces_volume_Cylinder_to_total,faces_volume_Plane_to_total,faces_volume_total,holes_count,holes_diameter_avg,holes_diameter_max,holes_diameter_min,holes_diameter_std,holes_diameters_count,holes_volume_avg,holes_volume_max,holes_volume_min,holes_volume_std,objects_ShapeType,objects_childshapes_count,objects_compounds,objects_count,objects_count_shells,objects_count_solids,objects_count_subshapes,objects_fullname,objects_haschild,objects_label,objects_lenght,shape_area_total,shape_volume_total,step_file,volume_boundbox,volume_part,volume_ratio_part_to_boundbox,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,file_name
C:/Users/BS/cpstdata/devis/2021/c0000028amplitudelisses/2021002520210212/pm2010818arefb40037049couvercle1/pm2010818a.step,210,44100,1.03635e+07,1.5435e+06,9.261e+06,1.15972e+07,1.72725e+06,257250,235,55225,1.93288e+06,1.29779e+07,287875,35,1225,42875,317.096,1.72725e+06,0.272256,19,228,217,464,6.86062,15.708,25.1975,246.315,0.0252656,2298.39,0.235756,7204.3,0.738978,464,233,21.0108,3.48787e-15,29.267,9749.01,110.029,204.204,554.344,4705.62,0.0263578,20687.5,0.115878,153135,0.857764,148,45875,1206.27,2.86335,4830.58,178528,9,85,54,148,856.265,1108.69,11898.4,20,47651.6,8926.41,3.6081,14942.7,2,2,6.25,20,53,7.4,1,13.4689,0.660708,0.339292,2.56952e-15,68632.9,18,6.61111,10,2.5,2.17022,7,402.641,703.246,41.7243,192.055,Solid,1,0,10,1,1,1,Unnamed93#Solid008,False,Solid008,19498,178528,470255,C:/Users/BS/cpstdata/devis/2021/c0000028amplit...,1.72725e+06,470255,0.272256,C:,Users,BS,cpstdata,devis,2021,c0000028amplitudelisses,2021002520210212,pm2010818arefb40037049couvercle1,pm2010818a.step,pm2010818a.step


,boundbox_X,boundbox_X2,boundbox_X2Y,boundbox_X2Z,boundbox_X3,boundbox_XY2,boundbox_XYZ,boundbox_XZ2,boundbox_Y,boundbox_Y2,boundbox_Y2Z,boundbox_Y3,boundbox_YZ2,boundbox_Z,boundbox_Z2,boundbox_Z3,boundbox_diagonal,boundbox_optimized_volume,boundbox_optimized_volume_ratio,edges_count_BSplineCurve,edges_count_Circle,edges_count_Line,edges_count_total,edges_lenght_25%,edges_lenght_50%,edges_lenght_75%,edges_lenght_BSplineCurve,edges_lenght_BSplineCurve_to_total,edges_lenght_Circle,edges_lenght_Circle_to_total,edges_lenght_Line,edges_lenght_Line_to_total,edges_lenght_count,edges_lenght_max,edges_lenght_mean,edges_lenght_min,edges_lenght_std,edges_lenght_total,faces_area_25%,faces_area_50%,faces_area_75%,faces_area_Cone,faces_area_Cone_to_total,faces_area_Cylinder,faces_area_Cylinder_to_total,faces_area_Plane,faces_area_Plane_to_total,faces_area_count,faces_area_max,faces_area_mean,faces_area_min,faces_area_std,faces_area_total,faces_count_Cone,faces_count_Cylinder,faces_count_Plane,faces_count_total,faces_orient_area_25%,faces_orient_area_50%,faces_orient_area_75%,faces_orient_area_count,faces_orient_area_max,faces_orient_area_mean,faces_orient_area_min,faces_orient_area_std,faces_orient_quantity_25%,faces_orient_quantity_50%,faces_orient_quantity_75%,faces_orient_quantity_count,faces_orient_quantity_max,faces_orient_quantity_mean,faces_orient_quantity_min,faces_orient_quantity_std,faces_volume_Cone_to_total,faces_volume_Cylinder_to_total,faces_volume_Plane_to_total,faces_volume_total,holes_count,holes_diameter_avg,holes_diameter_max,holes_diameter_min,holes_diameter_std,holes_diameters_count,holes_volume_avg,holes_volume_max,holes_volume_min,holes_volume_std,objects_ShapeType,objects_childshapes_count,objects_compounds,objects_count,objects_count_shells,objects_count_solids,objects_count_subshapes,objects_fullname,objects_haschild,objects_label,objects_lenght,shape_area_total,shape_volume_total,step_file,volume_boundbox,volume_part,volume_ratio_part_to_boundbox,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,file_name
C:/Users/BS/cpstdata/devis/2021/c0000028amplitudelisses/2021002520210212/pm2010818arefb40037049couvercle1/pm2010818a.step,210,44100,1.03635e+07,1.5435e+06,9.261e+06,1.15972e+07,1.72725e+06,257250,235,55225,1.93288e+06,1.29779e+07,287875,35,1225,42875,317.096,1.72725e+06,0.272256,19,228,217,464,6.86062,15.708,25.1975,246.315,0.0252656,2298.39,0.235756,7204.3,0.738978,464,233,21.0108,3.48787e-15,29.267,9749.01,110.029,204.204,554.344,4705.62,0.0263578,20687.5,0.115878,153135,0.857764,148,45875,1206.27,2.86335,4830.58,178528,9,85,54,148,856.265,1108.69,11898.4,20,47651.6,8926.41,3.6081,14942.7,2,2,6.25,20,53,7.4,1,13.4689,0.660708,0.339292,2.56952e-15,68632.9,18,6.61111,10,2.5,2.17022,7,402.641,703.246,41.7243,192.055,Compound,9,1,10,9,9,9,Unnamed93#Compound,False,PM20-10818-A,22061.4,178528,470255,C:/Users/BS/cpstdata/devis/2021/c0000028amplit...,1.72725e+06,470255,0.272256,C:,Users,BS,cpstdata,devis,2021,c0000028amplitudelisses,2021002520210212,pm2010818arefb40037049couvercle1,pm2010818a.step,pm2010818a.step


Done
Number of batch skipped:  0


In [22]:
df_features

,boundbox_X,boundbox_X2,boundbox_X2Y,boundbox_X2Z,boundbox_X3,boundbox_XY2,boundbox_XYZ,boundbox_XZ2,boundbox_Y,boundbox_Y2,boundbox_Y2Z,boundbox_Y3,boundbox_YZ2,boundbox_Z,boundbox_Z2,boundbox_Z3,boundbox_diagonal,boundbox_optimized_volume,boundbox_optimized_volume_ratio,edges_count_BSplineCurve,edges_count_Circle,edges_count_Line,edges_count_total,edges_lenght_25%,edges_lenght_50%,edges_lenght_75%,edges_lenght_BSplineCurve,edges_lenght_BSplineCurve_to_total,edges_lenght_Circle,edges_lenght_Circle_to_total,edges_lenght_Line,edges_lenght_Line_to_total,edges_lenght_count,edges_lenght_max,edges_lenght_mean,edges_lenght_min,edges_lenght_std,edges_lenght_total,faces_area_25%,faces_area_50%,faces_area_75%,faces_area_Cone,faces_area_Cone_to_total,faces_area_Cylinder,faces_area_Cylinder_to_total,faces_area_Plane,faces_area_Plane_to_total,faces_area_count,faces_area_max,faces_area_mean,faces_area_min,faces_area_std,faces_area_total,faces_count_Cone,faces_count_Cylinder,faces_count_Plane,faces_count_total,faces_orient_area_25%,faces_orient_area_50%,faces_orient_area_75%,faces_orient_area_count,faces_orient_area_max,faces_orient_area_mean,faces_orient_area_min,faces_orient_area_std,faces_orient_quantity_25%,faces_orient_quantity_50%,faces_orient_quantity_75%,faces_orient_quantity_count,faces_orient_quantity_max,faces_orient_quantity_mean,faces_orient_quantity_min,faces_orient_quantity_std,faces_volume_Cone_to_total,faces_volume_Cylinder_to_total,faces_volume_Plane_to_total,faces_volume_total,holes_count,holes_diameter_avg,holes_diameter_max,holes_diameter_min,holes_diameter_std,holes_diameters_count,holes_volume_avg,holes_volume_max,holes_volume_min,holes_volume_std,objects_ShapeType,objects_childshapes_count,objects_compounds,objects_count,objects_count_shells,objects_count_solids,objects_count_subshapes,objects_fullname,objects_haschild,objects_label,objects_lenght,shape_area_total,shape_volume_total,step_file,volume_boundbox,volume_part,volume_ratio_part_to_boundbox,path_0,path_1,path_2,path_3,path_4,path_5,path_6,path_7,path_8,path_9,file_name
0,8,64,416,416,512,338,338,338,6.5,42.25,274.625,274.625,274.625,6.5,42.25,274.625,12.1861,84.5,2.12884,10,24,20,54,0.821211,1.74413,4.80745,14.7578,0.0921156,114.759,0.716304,30.6931,0.19158,54,10.2102,2.96685,0.535987,2.89766,160.21,3.04862,4.46012,5.63686,32.9345,0.117354,203.3,0.724408,44.4085,0.158238,19,137.328,14.7707,2.14493,32.9144,280.643,8,2,9,19,11.1461,14.5866,30.3241,8,137.327,35.0804,8.9202,45.4065,1,2,4,8,4,2.375,1,1.40789,0.0905826,0.909417,2.77696e-16,202.788,2,4.871,6.5,3.242,1.629,2,147.359,234.936,59.7825,87.5768,Solid,1,0,10,1,1,1,Unnamed93#Solid,False,Solid,320.42,280.643,179.887,C:/Users/BS/cpstdata/devis/2021/c0000028amplit...,338,179.887,0.53221,C:,Users,BS,cpstdata,devis,2021,c0000028amplitudelisses,2021002520210212,pm2010818arefb40037049couvercle1,pm2010818a.step,pm2010818a.step
1,8,64,416,416,512,338,338,338,6.5,42.25,274.625,274.625,274.625,6.5,42.25,274.625,12.1861,84.5,2.12884,10,24,20,54,0.821211,1.74413,4.80745,14.7578,0.0921156,114.759,0.716304,30.6931,0.19158,54,10.2102,2.96685,0.535987,2.89766,160.21,3.04862,4.46012,5.63686,32.9345,0.117354,203.3,0.724408,44.4085,0.158238,19,137.328,14.7707,2.14493,32.9144,280.643,8,2,9,19,11.1461,14.5866,30.3241,8,137.327,35.0804,8.9202,45.4065,1,2,4,8,4,2.375,1,1.40789,0.0905826,0.909417,3.55835e-16,202.788,2,4.871,6.5,3.242,1.629,2,147.359,234.936,59.7825,87.5768,Solid,1,0,10,1,1,1,Unnamed93#Solid001,False,Solid001,320.42,280.643,179.887,C:/Users/BS/cpstdata/devis/2021/c0000028amplit...,338,179.887,0.53221,C:,Users,BS,cpstdata,devis,2021,c0000028amplitudelisses,2021002520210212,pm2010818arefb40037049couvercle1,pm2010818a.step,pm2010818a.step
2,8,64,416,416,512,338,338,338,6.5,42.25,274.625,274.625,274.625,6.5,42.25,274.625,12.1861,84.5,2.12884,10,24,20,54,0.821211,1.74413,4.80745,14.7578,0.0921156,114.759,0.716304,30.6931,0.19158,54,10.2102,2.96685,0.535987,2.89766,160.21,3.04862,4.46012,5.63686,32.9345,0.117354,203.3,0.